In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from glob import glob
import pickle

from PIL import ImageColor
import matplotlib.colors

from utils import FEATURES_DIR, RESULTS_DIR
from analyses.ridge_regression_decoding import NUM_CV_SPLITS, GLM_OUT_DIR, calc_rsa
from analyses.calc_noise_ceilings import RSA_NOISE_CEILING_DIR
from notebook_utils import load_results_data, add_avg_subject, COLORS_PLOT_CATEGORICAL

In [2]:
def calc_alt_rsas():
    results_root_dir = os.path.expanduser(f'~/data/multimodal_decoding/glm/')

    result_files = sorted(glob(f"{results_root_dir}/*/*/*/results.p"))
    for result_file_path in tqdm(result_files):        
        results = pickle.load(open(result_file_path, 'rb'))

        for metric in ["spearmanr", "pearsonr"]:
            for matrix_metric in ["spearmanr", "pearsonr"]:
                rsa = calc_rsa(results['predictions'], results['latents'], metric, matrix_metric)
                results[f"rsa_{metric}_{matrix_metric}"] = rsa

        pickle.dump(results, open(result_file_path, 'wb'))

# calc_alt_rsas()

In [3]:
RSA_METRIC = "spearmanr"
RSA_MATRIX_METRIC = "spearmanr"
all_data = load_results_data()
all_data

,alpha,model,subject,features,training_mode,testing_mode,best_val_acc,metric,value,best_val_loss,ensemble,fold,best_val_mse,model_feat
0,100000.0,bert-base-uncased,sub-01,lang,train,test,True,rsa,0.147229,NaN,NaN,NaN,NaN,bert-base-uncased_lang
1,100000.0,bert-base-uncased,sub-01,lang,train,test,True,pairwise_acc,0.861387,NaN,NaN,NaN,NaN,bert-base-uncased_lang
2,100000.0,bert-base-uncased,sub-01,lang,train,test,True,pairwise_acc_captions,0.860455,NaN,NaN,NaN,NaN,bert-base-uncased_lang
3,100000.0,bert-base-uncased,sub-01,lang,train,test,True,pairwise_acc_images,0.884472,NaN,NaN,NaN,NaN,bert-base-uncased_lang
4,100000.0,bert-base-uncased,sub-01,lang,train,test,True,rsa_spearmanr_spearmanr,0.147229,NaN,NaN,NaN,NaN,bert-base-uncased_lang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,100000.0,vit-b-16,sub-07,vision,train,test,True,pairwise_acc_images,0.890476,NaN,NaN,NaN,NaN,vit-b-16_vision
2008,100000.0,vit-l-16,sub-07,vision,train,test,True,rsa,0.177623,NaN,NaN,NaN,NaN,vit-l-16_vision
2009,100000.0,vit-l-16,sub-07,vision,train,test,True,pairwise_acc,0.851656,NaN,NaN,NaN,NaN,vit-l-16_vision
2010,100000.0,vit-l-16,sub-07,vision,train,test,True,pairwise_acc_captions,0.778468,NaN,NaN,NaN,NaN,vit-l-16_vision


In [4]:
base_palette = COLORS_PLOT_CATEGORICAL
base_palette = [matplotlib.colors.to_rgb(color) for color in base_palette]
# print(base_palette)
def model_feat_color_palette(data, model_feat_order):
    assert len(data.model.unique()) <= len(base_palette)
    model_to_color = {model: color for model, color in zip(data.model.unique(), base_palette)}
    
    new_palette = []
    prev_model = None
    feat_idx = 0
    for model_feat in model_feat_order:
        model = data[data.model_feat == model_feat].iloc[0].model
        if prev_model != model:
            prev_model = model
            feat_idx = 0
        new_color = model_to_color[model]
        new_color = [c + feat_idx*0.05 if c + feat_idx*0.1 <=1 else 1 for c in new_color]
        new_palette.append(new_color)
        feat_idx += 1
    return new_palette

In [5]:
MODEL_ORDER = ["random-flava", "resnet-18", "resnet-50", "resnet-152", "vit-b-16", "vit-l-16", "vit_l_16", "dino-base", "dino-large", "dino-giant", "bert-base-uncased", "bert-large-uncased", "gpt2-small", "gpt2-medium", "gpt2-large", "gpt2-xl", "llama2-7b", "mistral", "visualbert", "clip", "imagebind", "lxmert", "vilt", "flava"]

def calc_model_feat_order(data):
    all_model_feats = data.model_feat.unique()
    all_models = data.model.unique()
    for model in all_models:
        if model not in MODEL_ORDER:
            raise RuntimeError(f"Model missing in order: {model}")
    model_feat_order = []
    for model in MODEL_ORDER:
        for feats in ["vision", "lang", "concat"]:
            model_feat = f"{model}_{feats}"
            if model_feat in all_model_feats:
                model_feat_order.append(model_feat)

    return model_feat_order

In [6]:
distance_metric = 'cosine'
METRICS = [f'acc_{distance_metric}', f'acc_{distance_metric}_captions', f'acc_{distance_metric}_images']

def plot_metrics(data, x_variable="subject", order=None, hue_variable="model_feat", hue_order=None, row_variable=None, metrics=METRICS, ylim=(0.5, 1), add_avg_over_subjects=False, plot_legend=True, palette=None, noise_ceiling=None, hatches=None):
    data_filtered = data[data.metric.isin(metrics)]
    if add_avg_over_subjects:
        data_filtered = add_avg_subject(data_filtered)

    sns.set_style("ticks", {'axes.grid' : True})
    g = sns.catplot(data_filtered, kind="bar", x=x_variable, order=order, y="value", row=row_variable, col="metric", col_order=metrics, height=9, aspect=2, hue=hue_variable, hue_order=hue_order, palette=palette, err_kws={'linewidth': 0.5, 'alpha': 0.99})
    g._legend.remove()
    bbox_extra_artists = None
    if plot_legend:
        # lgd = g.fig.legend(loc='upper left', title="", bbox_to_anchor=(1, 0.9), ncol=2)
        lgd = g.fig.legend(loc='upper left', title="", bbox_to_anchor=(0.05, 0.95), ncol=9)
        bbox_extra_artists = (lgd,)

    if noise_ceiling is not None:
        g.axes[0, 0].axhline(y=noise_ceiling)

    if hatches is not None:
        for i, thisbar in enumerate(g.axes[0, 0].patches[:len(hatches)]):
            thisbar.set_hatch(hatches[i])

    g.set(ylim=ylim)
    
    # plt.suptitle("Test Performance", fontsize=16)
    plt.tight_layout()
    
    plt.savefig(os.path.join(RESULTS_DIR, f"{hue_variable}_comparison_{metrics[0]}.png"), bbox_extra_artists=bbox_extra_artists, bbox_inches='tight', dpi=300)
    return g, data_filtered

In [7]:
# def model_size_color_palette(data, model_order):
#     for model in data.model.unique():
#         if model not in model_order:
#             raise RuntimeError(f"Model missing in order: {model}")
#     data["model_prefix"] = data.model.apply(lambda name: name.split('-')[0] if '-' in name else name.split('_')[0] if '_' in name else name)
#     assert len(data.model_prefix.unique()) <= len(base_palette)
#     model_prefix_to_color = {model_prefix: color for model_prefix, color in zip(data.model_prefix.unique(), base_palette)}
    
#     new_palette = []
#     prev_model_prefix = None
#     feat_idx = 0
#     for model in model_order:
#         filtered = data[data.model == model]
#         if len(filtered) < 1:
#             raise RuntimeError(f"Model missing in data: {model}")
#             continue
#         model_prefix = filtered.iloc[0].model_prefix
#         if prev_model_prefix != model_prefix:
#             prev_model_prefix = model_prefix
#             feat_idx = 0
#         new_color = model_prefix_to_color[model_prefix]
#         new_color = [min(1, c + feat_idx*0.1) for c in new_color]
#         new_palette.append(new_color)
#         feat_idx += 1
#     return new_palette

In [8]:
FEATS_TO_HATCH = {"vision": '\\\\', "lang": '//', "concat": 'xx'}
def model_size_hatch_palette(data, model_feat_order):
    for model in data.model_feat.unique():
        if model not in model_feat_order:
            raise RuntimeError(f"Model missing in order: {model}")
    data["model_prefix"] = data.model.apply(lambda name: name.split('-')[0] if '-' in name else name.split('_')[0] if '_' in name else name)
    prefixes_ordered = set([name.split('-')[0] if '-' in name else name.split('_')[0] if '_' in name else name for name in model_feat_order])
    # print(prefixes_ordered)
    assert len(data.model_prefix.unique()) <= len(base_palette)
    model_prefix_to_color = {model_prefix: color for model_prefix, color in zip(prefixes_ordered, base_palette)}
    print({model: matplotlib.colors.to_hex(color) for model, color in model_prefix_to_color.items()})
    hatches = []
    new_palette = []
    prev_model_prefix = None
    prev_model = None
    suffix_idx = 2
    for model_feat in model_feat_order:
        model = data[data.model_feat == model_feat].iloc[0]
        if prev_model_prefix != model.model_prefix:
            prev_model_prefix = model.model_prefix
            suffix_idx = 2
        if prev_model != model.model:
            prev_model = model.model
            suffix_idx -= 1

        new_color = model_prefix_to_color[model.model_prefix]
        new_color = [max(0, min(1, c + suffix_idx*0.15)) for c in new_color]
        new_palette.append(new_color)
        hatches.append(FEATS_TO_HATCH[model.features])

    return new_palette, hatches

In [9]:
def plot_metrics_single_subject(data, x_variable="subject", order=None, hue_variable="model_feat", hue_order=None, metric="pairwise_acc", ylim=(0.5, 1), plot_legend=True, palette=None, noise_ceiling=None, hatches=None):
    data_filtered = data[data.metric == metric]

    sns.set_style("ticks", {'axes.grid' : True})
    g = sns.barplot(data_filtered, x=x_variable, order=order, y="value", hue=hue_variable, hue_order=hue_order, palette=palette, err_kws={'linewidth': 0.5}, width=0.98)
    g.legend().remove()
    bbox_extra_artists = None
    if plot_legend:
        # lgd = g.fig.legend(loc='upper left', title="", bbox_to_anchor=(1, 0.9), ncol=2)
        lgd = g.legend(loc='upper left', title="", bbox_to_anchor=(0.05, 0.95), ncol=9)
        bbox_extra_artists = (lgd,)

    if noise_ceiling is not None:
        g.axhline(y=noise_ceiling)

    if hatches is not None:
        for i, thisbar in enumerate(g.patches[:len(hatches)]):
            thisbar.set_hatch(hatches[i])

    g.set(ylim=ylim, ylabel=metric)
    
    # plt.suptitle("Test Performance", fontsize=16)
    plt.tight_layout()
    
    plt.savefig(os.path.join(RESULTS_DIR, f"{hue_variable}_comparison_{metric}.png"), bbox_extra_artists=bbox_extra_artists, bbox_inches='tight', dpi=300)
    return g, data_filtered

In [10]:
sns.set(rc={'figure.figsize':(25,10)})
data_to_plot = all_data
MODELS_EXCLUDED = []
data_to_plot = all_data[~all_data.model_feat.isin(MODELS_EXCLUDED)]
data_to_plot = data_to_plot[data_to_plot.best_val_acc == True]

data_to_plot_avg = add_avg_subject(data_to_plot)
data_to_plot_avg = data_to_plot_avg[data_to_plot_avg.subject == "average"]
model_feat_order = calc_model_feat_order(data_to_plot_avg)
model_size_palette, hatches = model_size_hatch_palette(data_to_plot_avg, model_feat_order)
g, data_plotted = plot_metrics_single_subject(data_to_plot_avg, metric="pairwise_acc", x_variable="subject", hue_variable="model_feat", hue_order=model_feat_order, palette=model_size_palette, hatches=hatches)


RuntimeError: Model missing in order: bridgetower-large

In [ ]:
data_to_plot = all_data
MODELS_EXCLUDED = []
data_to_plot = all_data[~all_data.model_feat.isin(MODELS_EXCLUDED)]
data_to_plot = data_to_plot[data_to_plot.best_val_acc == True]

data_to_plot_avg = add_avg_subject(data_to_plot)
data_to_plot_avg = data_to_plot_avg[data_to_plot_avg.subject == "average"]
model_feat_order = calc_model_feat_order(data_to_plot_avg)
model_size_palette, hatches = model_size_hatch_palette(data_to_plot_avg, model_feat_order)
g, data_plotted = plot_metrics_single_subject(data_to_plot_avg, metric="rsa", x_variable="subject", hue_variable="model_feat", hue_order=model_feat_order, palette=model_size_palette, hatches=hatches, ylim=(0, 0.3 ))

# g, data_plotted = plot_metrics(data_to_plot_avg, metrics=["rsa"], x_variable="subject", hue_variable="model_feat", hue_order=model_feat_order, palette=model_size_palette, hatches=hatches)


In [ ]:
# data_to_plot = all_data
# MODELS_EXCLUDED = []
# data_to_plot = all_data[~all_data.model_feat.isin(MODELS_EXCLUDED)]

# data_to_plot = data_to_plot[data_to_plot.best_val_acc == True]

# model_feat_order = calc_model_feat_order(data_to_plot)
# palette = model_feat_color_palette(data_to_plot, model_feat_order)

# g, data_plotted = plot_metrics(data_to_plot, metrics=["pairwise_acc"], palette=palette, hue_order=model_feat_order, add_avg_over_subjects=True)
# g, _ = plot_metrics(data_to_plot, metrics=["pairwise_acc_captions"], palette=palette, hue_order=model_feat_order, add_avg_over_subjects=True)
# g, _ = plot_metrics(data_to_plot, metrics=["pairwise_acc_images"], palette=palette, hue_order=model_feat_order, add_avg_over_subjects=True)


In [ ]:
rsa_noise_ceilings_path = os.path.join(RSA_NOISE_CEILING_DIR, f"{RSA_METRIC}_{RSA_MATRIX_METRIC}.p")
rsa_noise_ceilings = pickle.load(open(rsa_noise_ceilings_path, 'rb'))
rsa_noise_ceiling_mean = np.mean(list(rsa_noise_ceilings.values()))
rsa_noise_ceiling_std = np.std(list(rsa_noise_ceilings.values()))
print(rsa_noise_ceiling_mean)
print(rsa_noise_ceiling_std)
# g, _ = plot_metrics(data_to_plot, metrics=["rsa_spearmanr_spearmanr"], ylim=(0, 0.7), palette=palette, hue_order=model_feat_order, noise_ceiling=rsa_noise_ceiling_mean)
# g, _ = plot_metrics(data_to_plot, metrics=["rsa_spearmanr_spearmanr"], ylim=(0, 0.3), palette=palette, hue_order=model_feat_order)


In [ ]:
all_data[all_data.best_val_loss == True].alpha.unique()

In [ ]:

# data_to_print = all_data[(all_data.best_val_acc == True) & (all_data.ensemble != True)]
# data_to_print = data_to_print[data_to_print.subject.isin(["sub-01", "sub-02", "sub-03"])]
# # data_to_print = data_to_print[data_to_print.model_feat.str.contains('bert-large-uncased_lang')]
# data_to_print = data_to_print[data_to_print.model_feat.str.contains('vilt_concat')]

# print(len(data_to_print[data_to_print.metric == "pairwise_acc"].value))
# # assert len(data_to_print[data_to_print.metric == "acc_cosine"].value) == 6
# print(data_to_print[data_to_print.metric == "pairwise_acc"].value.mean())
# data_to_print[data_to_print.metric == "pairwise_acc"]

In [ ]:

# data_to_print = all_data[(all_data.best_val_mse == True) & (all_data.ensemble != True)]
# data_to_print = data_to_print[data_to_print.subject == "sub-01"]
# data_to_print = data_to_print[data_to_print.model_feat.str.contains('flava_lang')]
# print(len(data_to_print[data_to_print.metric == "pairwise_acc"].value))
# # assert len(data_to_print[data_to_print.metric == "acc_cosine"].value) == 6
# print(data_to_print[data_to_print.metric == "pairwise_acc"].value.mean())
# data_to_print[data_to_print.metric == "pairwise_acc"]